# Edge Cloud Joint Inference with Seldon Core and Tempo

Description

## Setup Environment

In [7]:
!conda env create --name edge-cloud-inference --file ./conda/edge-cloud-inference.yaml

In [6]:
!conda activate edge-cloud-inference

## Train models 

In [ ]:
!pip install git+https://github.com/SachinVarghese/tempo.git@tempo-k8s-nodename#egg=mlops-tempo&subdirectory=tempo

In [1]:
import os
from tempo.utils import logger
import logging
logger.setLevel(logging.ERROR)
logging.basicConfig(level=logging.ERROR)
ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"

In [21]:
# %load src/train.py
from src.data import IrisData
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib

EdgeModelFolder = "edge"
CloudModelFolder = "cloud"


def train_edge_model(data: IrisData, artifacts_folder: str):
    logreg = LogisticRegression(C=1e5)
    logreg.fit(data.X, data.y)
    with open(f"{artifacts_folder}/{EdgeModelFolder}/model.joblib", "wb") as f:
        joblib.dump(logreg, f)


def train_cloud_model(data: IrisData, artifacts_folder: str):
    clf = XGBClassifier()
    clf.fit(data.X, data.y)
    clf.save_model(f"{artifacts_folder}/{CloudModelFolder}/model.bst")


In [2]:
from src.data import IrisData
from src.train import train_edge_model, train_cloud_model
data = IrisData()
train_edge_model(data, ARTIFACTS_FOLDER)
train_cloud_model(data, ARTIFACTS_FOLDER)


/home/sachin/miniconda3/envs/edge-cloud-inference/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:55:17] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Create Tempo artifacts

In [7]:
# %load src/tempo.py
from typing import Tuple

import numpy as np
from src.train import CloudModelFolder, EdgeModelFolder

from tempo.serve.metadata import ModelFramework, RuntimeOptions, KubernetesOptions
from tempo.serve.model import Model
from tempo.serve.pipeline import Pipeline, PipelineModels
from tempo.serve.utils import pipeline

PipelineFolder = "joint-classifier"
EdgePredictionTag = "edge prediction"
CloudPredictionTag = "cloud prediction"

edgeKubernetesOptions = RuntimeOptions()
edgeKubernetesOptions.k8s_options = KubernetesOptions(
    replicas=1, 
    nodeName="edge-compute",
    namespace="production",
    authSecretName="minio-secret",
)


cloudKubernetesOptions = RuntimeOptions()
cloudKubernetesOptions.k8s_options = KubernetesOptions(
    replicas=2, 
    nodeName="gke-cloud-core-default-pool-6eddfeeb-71bg",
    namespace="production",
    authSecretName="minio-secret",
)

def get_tempo_artifacts(artifacts_folder: str) -> Tuple[Pipeline, Model, Model]:

    edge_model = Model(
        name="edge-model",
        platform=ModelFramework.SKLearn,
        local_folder=f"{artifacts_folder}/{EdgeModelFolder}",
        uri="s3://tempo/joint-inference/edge",
        description="An Edge based Iris classification model",
        runtime_options=edgeKubernetesOptions,
    )

    cloud_model = Model(
        name="cloud-model",
        platform=ModelFramework.XGBoost,
        local_folder=f"{artifacts_folder}/{CloudModelFolder}",
        uri="s3://tempo/joint-inference/cloud",
        description="An Cloud based Iris classification model",
        runtime_options=cloudKubernetesOptions,
    )

    @pipeline(
        name="joint-classifier",
        uri="s3://tempo/basic/pipeline",
        local_folder=f"{artifacts_folder}/{PipelineFolder}",
        models=PipelineModels(edge_inference=edge_model,
                              cloud_inference=cloud_model),
        description="A pipeline to make an edge based prediction or cloud based joint prediction for Iris classification",
        runtime_options=cloudKubernetesOptions,
    )
    def classifier(payload: np.ndarray) -> Tuple[np.ndarray, str]:
        # Custom Logic for hard example mining based on threshold, IBT, Cross Entropy etc
        res1 = classifier.models.edge_inference(input=payload)
        if res1[0] == 1:
            return res1, EdgePredictionTag
        else:
            return classifier.models.cloud_inference(input=payload), CloudPredictionTag

    return classifier, edge_model, cloud_model


In [8]:
from src.tempo import get_tempo_artifacts
classifier, edge_model, cloud_model = get_tempo_artifacts(ARTIFACTS_FOLDER)

# Save Classifier

In [4]:
from tempo.serve.loader import save
save(classifier)

Packing environment at '/home/sachin/miniconda3/envs/tempo-b93dd311-0858-42af-9cee-170b2db19559' to '/home/sachin/projects/mlops/edge-cloud-inference/artifacts/joint-classifier/environment.tar.gz'
[########################################] | 100% Completed | 28.2s


# Deploy to Kubernetes

In [24]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [25]:
from tempo.serve.loader import upload
upload(edge_model)
upload(cloud_model)
upload(classifier)

In [13]:
!kubectl create ns production
!kubectl apply -f src/rbac -n production

secret/minio-secret created
serviceaccount/tempo-pipeline created
role.rbac.authorization.k8s.io/tempo-pipeline created
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding created


In [9]:
from tempo.serve.metadata import KubernetesOptions
from tempo.seldon.k8s import SeldonCoreOptions
runtime_options = SeldonCoreOptions()

In [6]:
from tempo import deploy
remote_model = deploy(classifier, options=runtime_options)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Audit-Id': 'a4d272c1-552b-4846-b296-aff68877d07d', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Tue, 13 Jul 2021 12:59:30 GMT', 'Content-Length': '288'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"seldondeployments.machinelearning.seldon.io \"joint-classifier\" not found","reason":"NotFound","details":{"name":"joint-classifier","group":"machinelearning.seldon.io","kind":"seldondeployments"},"code":404}



In [36]:
import numpy as np

print(remote_model.predict(payload=np.array([[0, 0, 0, 0]])))
print(remote_model.predict(payload=np.array([[1, 2, 3, 4]])))

In [16]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
k8s_runtime = SeldonKubernetesRuntime(runtime_options)
models = k8s_runtime.list_models(namespace="production")
print("Name\t\tDescription")
for model in models:
    details = model.get_tempo().model_spec.model_details
    print(f"{details.name}\t{details.description}")

models[2].predict(payload=np.array([[1, 2, 3, 4]]))


Name		Description
cloud-model	An Cloud based Iris classification model
edge-model	An Edge based Iris classification model
joint-classifier	A pipeline to make an edge based prediction or cloud based joint prediction for Iris classification


{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'cloud prediction'}

In [35]:
remote_model.undeploy()

INFO:tempo:Undeploying joint-classifier
DEBUG:tempo:Loading external kubernetes config
INFO:tempo:Undeploying edge-model
DEBUG:tempo:Loading external kubernetes config
INFO:tempo:Undeploying cloud-model
DEBUG:tempo:Loading external kubernetes config
